In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()
think3=test_data.head()
think3.loc[think3.Sex=='male','Sex']=1
print(think3)
test_data.Embarked.unique()

   PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James       1   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis       1   
3          895       3                              Wirz, Mr. Albert       1   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S  


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


array(['Q', 'S', 'C'], dtype=object)

In [4]:
train_data.Age.mean()

29.69911764705882

In [5]:
print(train_data.isnull().sum())
print(train_data.Age.isnull().sum())
print("train data mean is:",(train_data.Age.mean())/1)
train_data.loc[train_data.Age.isnull(),'Age']=train_data.Age.mean() #Age 결측 평균채우기
print(train_data)
train_data.loc[train_data.Embarked.isnull(),'Embarked']='S' #Embarked 결측 최빈값 채우기
print(train_data.isnull().sum())

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
177
train data mean is: 29.69911764705882
     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex        Age  \
0                              Braund, Mr. Owen Harris    male  22.000000   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.000000   
2                            

In [6]:
train_data.loc[train_data.Sex=='female','Sex']=1
train_data.loc[train_data.Sex=='male','Sex']=0
train_data.loc[train_data.Embarked=='S','Embarked']=0
train_data.loc[train_data.Embarked=='Q','Embarked']=1
train_data.loc[train_data.Embarked=='C','Embarked']=2
train_data #text data인 Sex, Embarked 를 측정 가능한 숫자로 바꿈.
train_data.Embarked.unique()

array([0, 2, 1], dtype=object)

In [7]:
x=[]
for i, rows in train_data.iterrows():
    x.append([
    rows['PassengerId'],
    rows['Pclass'],
    #rows['Name'], #string
    rows['Sex'], #string
    rows['Age'],
    rows['Parch'],
    #rows['Ticket'], #string
    #rows['Fare'], #string
    rows['Embarked']] #string
    )
print(x)
y=train_data['Survived'].values

[[1, 3, 0, 22.0, 0, 0], [2, 1, 1, 38.0, 0, 2], [3, 3, 1, 26.0, 0, 0], [4, 1, 1, 35.0, 0, 0], [5, 3, 0, 35.0, 0, 0], [6, 3, 0, 29.69911764705882, 0, 1], [7, 1, 0, 54.0, 0, 0], [8, 3, 0, 2.0, 1, 0], [9, 3, 1, 27.0, 2, 0], [10, 2, 1, 14.0, 0, 2], [11, 3, 1, 4.0, 1, 0], [12, 1, 1, 58.0, 0, 0], [13, 3, 0, 20.0, 0, 0], [14, 3, 0, 39.0, 5, 0], [15, 3, 1, 14.0, 0, 0], [16, 2, 1, 55.0, 0, 0], [17, 3, 0, 2.0, 1, 1], [18, 2, 0, 29.69911764705882, 0, 0], [19, 3, 1, 31.0, 0, 0], [20, 3, 1, 29.69911764705882, 0, 2], [21, 2, 0, 35.0, 0, 0], [22, 2, 0, 34.0, 0, 0], [23, 3, 1, 15.0, 0, 1], [24, 1, 0, 28.0, 0, 0], [25, 3, 1, 8.0, 1, 0], [26, 3, 1, 38.0, 5, 0], [27, 3, 0, 29.69911764705882, 0, 2], [28, 1, 0, 19.0, 2, 0], [29, 3, 1, 29.69911764705882, 0, 1], [30, 3, 0, 29.69911764705882, 0, 0], [31, 1, 0, 40.0, 0, 2], [32, 1, 1, 29.69911764705882, 0, 2], [33, 3, 1, 29.69911764705882, 0, 1], [34, 2, 0, 66.0, 0, 0], [35, 1, 0, 28.0, 0, 2], [36, 1, 0, 42.0, 0, 0], [37, 3, 0, 29.69911764705882, 0, 2], [38, 3,

In [8]:
print(np.array(x).shape)
print(np.array(y).shape)

(891, 6)
(891,)


In [9]:
import tensorflow as tf

model=tf.keras.models.Sequential([
    tf.keras.layers.Dense(64,activation='tanh'),
    tf.keras.layers.Dense(128,activation='tanh'),
    tf.keras.layers.Dense(1,activation='sigmoid'),
])

model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
think=model.fit(np.array(x),np.array(y),epochs=500)

Epoch 1/500
28/28 [==============================] - 1s 1ms/step - loss: 0.6776 - accuracy: 0.6258
Epoch 2/500
28/28 [==============================] - 0s 1ms/step - loss: 0.6563 - accuracy: 0.6349
Epoch 3/500
28/28 [==============================] - 0s 1ms/step - loss: 0.6715 - accuracy: 0.6016
Epoch 4/500
28/28 [==============================] - 0s 1ms/step - loss: 0.6579 - accuracy: 0.6395
Epoch 5/500
28/28 [==============================] - 0s 1ms/step - loss: 0.6608 - accuracy: 0.6262
Epoch 6/500
28/28 [==============================] - 0s 1ms/step - loss: 0.6417 - accuracy: 0.6385
Epoch 7/500
28/28 [==============================] - 0s 1ms/step - loss: 0.6370 - accuracy: 0.6482
Epoch 8/500
28/28 [==============================] - 0s 1ms/step - loss: 0.5997 - accuracy: 0.7084
Epoch 9/500
28/28 [==============================] - 0s 1ms/step - loss: 0.5895 - accuracy: 0.7009
Epoch 10/500
28/28 [==============================] - 0s 1ms/step - loss: 0.5406 - accuracy: 0.7430
Epoch 11/

In [10]:
#test data에 대해서도 text 1 결측값 처리 2 숫자로 바꾸기 3 list형으로 바꾸기 함.
#1 결측값 처리
test_data.loc[test_data.Age.isnull(),'Age']=test_data.Age.mean() #Age 결측 평균채우기
print(test_data)
test_data.loc[test_data.Fare.isnull(),'Fare']=test_data.Fare.mean() #Embarked 결측 최빈값 채우기
print(test_data.isnull().sum())

     PassengerId  Pclass                                          Name  \
0            892       3                              Kelly, Mr. James   
1            893       3              Wilkes, Mrs. James (Ellen Needs)   
2            894       2                     Myles, Mr. Thomas Francis   
3            895       3                              Wirz, Mr. Albert   
4            896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)   
..           ...     ...                                           ...   
413         1305       3                            Spector, Mr. Woolf   
414         1306       1                  Oliva y Ocana, Dona. Fermina   
415         1307       3                  Saether, Mr. Simon Sivertsen   
416         1308       3                           Ware, Mr. Frederick   
417         1309       3                      Peter, Master. Michael J   

        Sex       Age  SibSp  Parch              Ticket      Fare Cabin  \
0         1  34.50000      0      0 

In [11]:
#2 숫자로 바꾸기
test_data.loc[test_data.Sex=='female','Sex']=1
test_data.loc[test_data.Sex=='male','Sex']=0
test_data.loc[test_data.Embarked=='S','Embarked']=0
test_data.loc[test_data.Embarked=='Q','Embarked']=1
test_data.loc[test_data.Embarked=='C','Embarked']=2
#text data인 Sex, Embarked 를 측정 가능한 숫자로 바꿈.
#3
testx=[]
for i, rows in test_data.iterrows():
    testx.append([
    rows['PassengerId'],
    rows['Pclass'],
    #rows['Name'], #string
    rows['Sex'], #string
    rows['Age'],
    rows['Parch'],
    #rows['Ticket'], #string
    #rows['Fare'], #string
    rows['Embarked']] #string
    )

In [12]:
test_data.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

In [13]:
submitdata2={'PassengerId':[],'Survived':[]}
submitdata2['PassengerId']=test_data['PassengerId']
예측값=model.predict(testx)
intpredict=np.around(예측값).astype(int)
for i,rows in test_data.iterrows():
    submitdata2['Survived'].append(intpredict[i][0])
submitdf2=pd.DataFrame(submitdata2)
submitdf2.to_csv("/kaggle/working/Submitdf.csv",mode='w',index=False)
print(submitdf2)

     PassengerId  Survived
0            892         0
1            893         0
2            894         1
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]
